<a href="https://colab.research.google.com/github/cowsilver57/sessac_test/blob/main/11%EC%9B%94_29%EC%9D%BC_(%EC%88%98)_Day_60.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#이전 레이어에서의 input이 (192,100,100)이라고 가정
#각 branch를 지날 때, image size가 바뀌지 않아야 함.
#Pooling을 지나도 image size가 바뀌지 않아야 함.

input_tensor = torch.randn(size=(192,100,100))

branch1 = ConvBlock(in_channels=192, out_channels=64, kernel_size=1)
out_branch1 = branch1(input_tensor)
print(out_branch1.shape)

branch2 = ConvBlock(in_channels=192, out_channels=128, kernel_size=3, padding=1)
out_branch2 = branch2(input_tensor)
print(out_branch2.shape)

branch3 = ConvBlock(in_channels=192, out_channels=32,kernel_size=5, padding=2)
out_branch3 = branch3(input_tensor)
print(out_branch3.shape)

branch4 = nn.MaxPool2d(kernel_size=3, padding=1, stride=1)
out_branch4 = branch4(input_tensor)
print(out_branch4.shape)

out_inception = torch.concat([out_branch1, out_branch2, out_branch3, out_branch4],axis=0)
print(out_inception.shape)

torch.Size([64, 100, 100])
torch.Size([128, 100, 100])
torch.Size([32, 100, 100])
torch.Size([192, 100, 100])
torch.Size([416, 100, 100])


# 1. Naive Version GoogLeNet

In [6]:
class InceptionNaive(nn.Module):
    def __init__(self,in_channels, ch1x1, ch3x3, ch5x5):
        super(InceptionNaive, self).__init__()
        self.branch1 = ConvBlock(in_channels=192, out_channels=ch1x1, kernel_size=1)
        self.branch2 = ConvBlock(in_channels=192, out_channels=ch3x3, kernel_size=3, padding=1)
        self.branch3 = ConvBlock(in_channels=192, out_channels=ch5x5,kernel_size=5, padding=2)
        self.branch4 = nn.MaxPool2d(kernel_size=3, padding=1, stride=1)

    def forward(self,x):
        out_branch1 = self.branch1(x)
        out_branch2 = self.branch2(x)
        out_branch3 = self.branch3(x)
        out_branch4 = self.branch4(x)
        out_inception = torch.concat([out_branch1, out_branch2, out_branch3, out_branch4],axis=0)
        return out_inception


inception_model = InceptionNaive(192,64,128,32)
input_tensor = torch.randn(size=(192, 100, 100))
output_tensor = inception_model(input_tensor)

print(output_tensor.shape)

torch.Size([416, 100, 100])


# 2. Inception module with dimension reductions

In [7]:
in_channels = 192
input_tensor = torch.randn(size=(in_channels, 100, 100))

#branch2
#red = reduction
ch3X3red, ch3X3 = 96,128
branch2 = nn.Sequential(
    ConvBlock(in_channels=in_channels, out_channels=ch3X3red, kernel_size=1),
    ConvBlock(in_channels=ch3X3red, out_channels=ch3X3, kernel_size=3, padding=1)
)
out_branch2 = branch2(input_tensor)

#branch3
ch5x5red, ch5x5 = 16, 32
branch3 = nn.Sequential(
    ConvBlock(in_channels=in_channels, out_channels=ch5x5red, kernel_size=1),
    ConvBlock(in_channels=ch5x5red, out_channels=ch5x5, kernel_size=5, padding=2)
)
out_branch3 = branch3(input_tensor)

#branch4
pool_proj = 32
branch4 = nn.Sequential(
    nn.MaxPool2d(kernel_size=3, padding=1, stride=1),
    ConvBlock(in_channels=in_channels, out_channels=pool_proj, kernel_size=1)
)
out_branch4 = branch4(input_tensor)

In [4]:
import torch
import torch.nn as nn

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0, stride=1):
        super(ConvBlock, self).__init__()

        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels,
                      kernel_size=kernel_size, padding=padding, stride=stride)
        self.relu = nn.ReLU()


    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        return x

In [12]:
import torch
import torch.nn as nn
class Inception(nn.Module):
    def __init__(self, in_channels, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj):
        super(Inception, self).__init__()

        self.branch1 = ConvBlock(in_channels=in_channels, out_channels=ch1x1, kernel_size=1)

        self.branch2 = nn.Sequential(
            ConvBlock(in_channels=in_channels, out_channels=ch3X3red, kernel_size=1),
            ConvBlock(in_channels=ch3X3red, out_channels=ch3X3, kernel_size=3, padding=1))

        self.branch3 = nn.Sequential(
            ConvBlock(in_channels=in_channels, out_channels=ch5x5red, kernel_size=1),
            ConvBlock(in_channels=ch5x5red, out_channels=ch5x5, kernel_size=5, padding=2))

        self.branch4 = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, padding=1, stride=1),
            ConvBlock(in_channels=in_channels, out_channels=pool_proj, kernel_size=1, padding=0, stride=1))

    def forward(self,x):
        out_branch1 = self.branch1(x)
        out_branch2 = self.branch2(x)
        out_branch3 = self.branch3(x)
        out_branch4 = self.branch4(x)
        out_inception = torch.concat([out_branch1, out_branch2, out_branch3, out_branch4],axis=1)
        return out_inception

BATCH_SIZE=8
inception_model = Inception(192,64,96,128,16,32,32)
input_tensor = torch.randn(size=(BATCH_SIZE,192, 100, 100))
output_tensor = inception_model(input_tensor)

print(output_tensor.shape)

torch.Size([8, 256, 100, 100])


In [18]:
class GoogLeNet(nn.Module):
    def __init__(self):
        super(GoogLeNet, self).__init__()

        self.conv1 = ConvBlock(in_channels=3, out_channels=64, kernel_size=7, stride=2, padding=3)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.conv2 = ConvBlock(in_channels=64, out_channels=192, kernel_size=3, stride=1, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception3a = Inception(in_channels=192, ch1x1=64, ch3x3red=96, ch3x3=128, ch5x5red=16, ch5x5=32, pool_proj=32)
        self.inception3b = Inception(in_channels=256, ch1x1=128, ch3x3red=128, ch3x3=192, ch5x5red=32, ch5x5=96, pool_proj=64)
        self.maxpool3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception4a = Inception(in_channels=480, ch1x1=192, ch3x3red=96, ch3x3=208, ch5x5red=16, ch5x5=48, pool_proj=64)
        self.inception4b = Inception(in_channels=512, ch1x1=160, ch3x3red=112, ch3x3=224, ch5x5red=24, ch5x5=64, pool_proj=64)
        self.inception4c = Inception(in_channels=512, ch1x1=128, ch3x3red=128, ch3x3=256, ch5x5red=24, ch5x5=64, pool_proj=64)
        self.inception4d = Inception(in_channels=512, ch1x1=112, ch3x3red=144, ch3x3=288, ch5x5red=32, ch5x5=64, pool_proj=64)
        self.inception4e = Inception(in_channels=528, ch1x1=256, ch3x3red=160, ch3x3=320, ch5x5red=32, ch5x5=128, pool_proj=128)
        self.maxpool4 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.inception5a = Inception(in_channels=832, ch1x1=256, ch3x3red=160, ch3x3=320, ch5x5red=32, ch5x5=128, pool_proj=128)
        self.inception5b = Inception(in_channels=832, ch1x1=384, ch3x3red=192, ch3x3=384, ch5x5red=48, ch5x5=128, pool_proj=128)

        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1)
        self.dropout = nn.Dropout(p=0.4)
        self.fc = nn.Linear(1024, 1000)

    def forward(self, x):
        x = self.conv1(x)
        print(x.shape)
        x = self.maxpool1(x)
        print(x.shape)
        x = self.conv2(x)
        print(x.shape)
        x = self.maxpool2(x)
        x = self.inception3a(x)
        x = self.inception3b(x)
        x = self.maxpool3(x)
        x = self.inception4a(x)
        x = self.inception4b(x)
        x = self.inception4c(x)
        x = self.inception4d(x)
        x = self.inception4e(x)
        x = self.maxpool4(x)
        x = self.inception5a(x)
        x = self.inception5b(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        x = self.fc(x)
        print(x.shape)
        return x

BATCH_SIZE = 32
input_tensor = torch.randn(size=(BATCH_SIZE,3, 223, 223))
model = GoogLeNet()
result = model(input_tensor)

torch.Size([32, 64, 112, 112])
torch.Size([32, 64, 56, 56])
torch.Size([32, 192, 56, 56])
torch.Size([32, 1000])
